# Orlando Criminal Activities

##### Presented by: Clark Jones, Umapathy Beema, Richard Morand <br> Dr. Spencer Lyon - Spring 2024 


Ojective: We will be exploring the criminal acts in Orlando, and create a model to showcase the types of activities done in the city. <br>

__Tools__ <br>
Collaboration: VSCode, Github, Discord <br>
Languages: Python, Julia
Packages/Libraries: Agents, OpenStreetMap, Pandas, etc.

In [2]:
#import packages here
import Pkg
Pkg.add(["OpenStreetMapX","CSV", "DataFrames"])

   Resolving package versions...
  No Changes to `C:\Users\umapa\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\umapa\.julia\environments\v1.10\Manifest.toml`


In [3]:
using Agents, OpenStreetMapX
using  CSV, DataFrames

In [60]:
# load and analyze data
data = CSV.read("Data/OPD_Officer-Involved_Shootings_20241125.csv", DataFrame)

Row,Case #,Date,Address,Incident Type,Number of Officers Involved,Officer Name,Officer Race,Ethnicity,Officer Gender,Officer Age,Officer Tenure,Number of Suspects Involved,Suspect Name,Suspect Race,Suspect Gender,Suspect Age,Suspect's Weapon,Suspect Hit,Fatal,Officer Hit,Fatal (Officer),Review Letter,Georeferenced Column
,String15,String15,String,String,Int64?,String?,String?,String31?,String31,String,String,Int64,String,String31,String7,String15,String31,String15,String15,String,String,String?,String
1,22-174307,05/18/2022,Mercy Drive and W.D. Judge Drive,Vehicle Stop,2,Marsy's Law Exempt,White/White,missing,"M,F",Marsy's Law Exempt,Marsy's Law Exempt,1,Carlos Delano Dafill Roberts Jr.,Black,M,29,Handgun,Yes,No,"No, No","No, No",https://www.orlando.gov/files/sharedassets/public/v/1/departments/public-safety/opd/open-data-docs/22-174307.pdf,POINT (-81.430522027 28.559823981)
2,18-448868,11/21/2018,9938 Universal Boulevard,Stolen Vehicle,1,"Chase, Alex",W,N,M,35,11,1,"Martinez Gonzalez, Joselito",White,M,30,Motor Vehicle,Yes,No,No,No,https://www1.cityoforlando.net/opendatadocs/saoletters/18-448869.pdf,POINT (-81.444407991 28.426330019)
3,18-219116,06/04/2018,4629 Cason Cove Drive,Attempted Suicide,1,"Sayer,Ryan",W,N,M,25,2,1,"White, Pamel",Black,F,63,Handgun,No,No,No,No,https://www1.cityoforlando.net/opendatadocs/saoletters/18-219116.pdf,POINT (-81.436934015 28.497201986)
4,20-397311,12/16/2020,"5434 Baldwin Park Street, Unit 101",Domestic Disturbance,1,missing,W,missing,M,40,14,1,"Robertson, Adam",White,M,34,Handgun,Yes,Yes,No,No,https://www.orlando.gov/files/sharedassets/public/departments/public-safety/opd/open-data-docs/ois_sao_zambito_ir-20-75.pdf,POINT (-81.31615297 28.574676001)
5,10-503738,10/26/2010,Hiawassee & SR 50,Armed Robbery,3,"Hinkles, Cedrick; Herb, Todd; Fischer, Bryan","B, W, W","N, N, N","M, M, M","30, 38, 37","6, 13, 4",1,"Soman, Acrifa",Black,M,21,Handgun,Yes,Yes,"No, No, No","No, No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/10-503738.pdf,POINT (-81.475974825 28.552001109)
6,13-022948,01/16/2013,1327 Arlington St,Drug Violation,2,"Mongeluzzo, Anthony; Bigelow, Chris","W, W","N, N","M, M","34, 29","6, 6",2,"Williams, Charles; Gamble, Karvas","Black, Black","M, M","37, 19",Handgun,"Yes, Yes","No, Yes","No, No","No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/13-022948.pdf,POINT (-81.398586976 28.552194982)
7,16-021812,01/16/2016,5400 North Lane,Traffic Stop,1,"Mills, Jonathan",W,N,M,33,4,1,"Lamons, Trevon",Black,M,17,Motor Vehicle,Yes,No,No,No,http://www1.cityoforlando.net/opendatadocs/saoletters/16-021812.pdf,POINT (-81.454123826 28.596308986)
8,14-086291,02/24/2014,595 W. Church St,Stalking,3,"Kipp, Alexander; Watts, Anthony; Johnston, David","W, B, W","N, N, N","M, M, M","32, 44, 24","1, 1, 1",1,"Lattimore, Derrick",Black,M,27,Motor Vehicle,No,No,"No, No, No","No, No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/14-086291.pdf,POINT (-81.386192991 28.540632985)
9,15-291159,07/17/2015,1657 Watauga Way,Suspicious Person,1,"Vidro, Kelvin",W,H,M,29,0.9,1,"Davis, Albert",Black,M,23,Hands/ASP,Yes,Yes,No,No,http://www1.cityoforlando.net/opendatadocs/saoletters/15-291159.pdf,POINT (-81.335092645 28.522822186)


In [5]:
# Function to convert Georeferenced Columm to Tuple 

function convert_to_tuple(str)
    test = str[7:end]
    test = strip(test, ['(',')'])
    test = split(test, ' ')

    return  (parse(Float64,test[1]), parse(Float64,test[2]))

end

convert_to_tuple (generic function with 1 method)

In [6]:
#Algorithms used
location = convert_to_tuple.(data[: , "Georeferenced Column"])

78-element Vector{Tuple{Float64, Float64}}:
 (-81.430522027, 28.559823981)
 (-81.444407991, 28.426330019)
 (-81.436934015, 28.497201986)
 (-81.31615297, 28.574676001)
 (-81.475974825, 28.552001109)
 (-81.398586976, 28.552194982)
 (-81.454123826, 28.596308986)
 (-81.386192991, 28.540632985)
 (-81.335092645, 28.522822186)
 (-81.444618041, 28.497712989)
 ⋮
 (-81.397620962, 28.544747996)
 (-81.307727984, 28.529062019)
 (-81.389041531, 28.538220076)
 (-81.429694969, 28.611062074)
 (-81.462643153, 28.490099922)
 (-81.269478007, 28.420598013)
 (-81.306244974, 28.518123007)
 (-81.388922977, 28.534726006)
 (-81.413406013, 28.601738987)

In [7]:
minmax(location[1], location[2])

((-81.444407991, 28.426330019), (-81.430522027, 28.559823981))

In [8]:
size(data)

(78, 23)

In [9]:
suspect_gender_cnt = combine(groupby(data,:"Suspect Gender"),nrow => :count)

Row,Suspect Gender,count
,String7,Int64
1,M,68
2,F,4
3,"M, M",4
4,"M, M, M",1
5,Unk,1


In [23]:
data.gender = data[!,"Suspect Gender"] . =  "M"

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\umapa\Documents\git\CatchingCrooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X26sZmlsZQ==.jl:1:39
data.gender = data[!,"Suspect Gender"] . =  "M"
#                                     ╙ ── whitespace is not allowed here

In [10]:
suspects_weapon_cnt = combine(groupby(data,:"Suspect's Weapon"),nrow => :count)

Row,Suspect's Weapon,count
,String31,Int64
1,Handgun,40
2,Motor Vehicle,15
3,Hands/ASP,1
4,Knife,2
5,No,3
6,"Firearm, Firearm, Unknown",1
7,"Handgun, None",1
8,Yes,1
9,Firearm,4


In [58]:
data.weap_cat = [Weapon == "Handgun" ? "Gun" : "No Weapon" for Weapon in data."Suspect's Weapon"]
data

Row,Case #,Date,Address,Incident Type,Number of Officers Involved,Officer Name,Officer Race,Ethnicity,Officer Gender,Officer Age,Officer Tenure,Number of Suspects Involved,Suspect Name,Suspect Race,Suspect Gender,Suspect Age,Suspect's Weapon,Suspect Hit,Fatal,Officer Hit,Fatal (Officer),Review Letter,Georeferenced Column,Point,gender,Match,weap_cat
,String15,String15,String,String,Int64?,String?,String?,String31?,String31,String,String,Int64,String,String31,Char,String15,String,String15,String15,String,String,String?,String,Any,Bool,Bool,String
1,22-174307,05/18/2022,Mercy Drive and W.D. Judge Drive,Vehicle Stop,2,Marsy's Law Exempt,White/White,missing,"M,F",Marsy's Law Exempt,Marsy's Law Exempt,1,Carlos Delano Dafill Roberts Jr.,Black,M,29,Handgun,Yes,No,"No, No","No, No",https://www.orlando.gov/files/sharedassets/public/v/1/departments/public-safety/opd/open-data-docs/22-174307.pdf,POINT (-81.430522027 28.559823981),"LatLon(lat=28.559823981°, lon=-81.430522027°)",false,true,Gun
2,18-448868,11/21/2018,9938 Universal Boulevard,Stolen Vehicle,1,"Chase, Alex",W,N,M,35,11,1,"Martinez Gonzalez, Joselito",White,M,30,Handgun,Yes,No,No,No,https://www1.cityoforlando.net/opendatadocs/saoletters/18-448869.pdf,POINT (-81.444407991 28.426330019),"LatLon(lat=28.426330019°, lon=-81.444407991°)",false,false,Gun
3,18-219116,06/04/2018,4629 Cason Cove Drive,Attempted Suicide,1,"Sayer,Ryan",W,N,M,25,2,1,"White, Pamel",Black,M,63,Handgun,No,No,No,No,https://www1.cityoforlando.net/opendatadocs/saoletters/18-219116.pdf,POINT (-81.436934015 28.497201986),"LatLon(lat=28.497201986°, lon=-81.436934015°)",false,true,Gun
4,20-397311,12/16/2020,"5434 Baldwin Park Street, Unit 101",Domestic Disturbance,1,missing,W,missing,M,40,14,1,"Robertson, Adam",White,M,34,Handgun,Yes,Yes,No,No,https://www.orlando.gov/files/sharedassets/public/departments/public-safety/opd/open-data-docs/ois_sao_zambito_ir-20-75.pdf,POINT (-81.31615297 28.574676001),"LatLon(lat=28.574676001°, lon=-81.31615297°)",false,true,Gun
5,10-503738,10/26/2010,Hiawassee & SR 50,Armed Robbery,3,"Hinkles, Cedrick; Herb, Todd; Fischer, Bryan","B, W, W","N, N, N","M, M, M","30, 38, 37","6, 13, 4",1,"Soman, Acrifa",Black,M,21,Handgun,Yes,Yes,"No, No, No","No, No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/10-503738.pdf,POINT (-81.475974825 28.552001109),"LatLon(lat=28.552001109°, lon=-81.475974825°)",false,true,Gun
6,13-022948,01/16/2013,1327 Arlington St,Drug Violation,2,"Mongeluzzo, Anthony; Bigelow, Chris","W, W","N, N","M, M","34, 29","6, 6",2,"Williams, Charles; Gamble, Karvas","Black, Black",M,"37, 19",Handgun,"Yes, Yes","No, Yes","No, No","No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/13-022948.pdf,POINT (-81.398586976 28.552194982),"LatLon(lat=28.552194982°, lon=-81.398586976°)",false,true,Gun
7,16-021812,01/16/2016,5400 North Lane,Traffic Stop,1,"Mills, Jonathan",W,N,M,33,4,1,"Lamons, Trevon",Black,M,17,Handgun,Yes,No,No,No,http://www1.cityoforlando.net/opendatadocs/saoletters/16-021812.pdf,POINT (-81.454123826 28.596308986),"LatLon(lat=28.596308986°, lon=-81.454123826°)",false,false,Gun
8,14-086291,02/24/2014,595 W. Church St,Stalking,3,"Kipp, Alexander; Watts, Anthony; Johnston, David","W, B, W","N, N, N","M, M, M","32, 44, 24","1, 1, 1",1,"Lattimore, Derrick",Black,M,27,Handgun,No,No,"No, No, No","No, No, No",http://www1.cityoforlando.net/opendatadocs/saoletters/14-086291.pdf,POINT (-81.386192991 28.540632985),"LatLon(lat=28.540632985°, lon=-81.386192991°)",false,false,Gun
9,15-291159,07/17/2015,1657 Watauga Way,Suspicious Person,1,"Vidro, Kelvin",W,H,M,29,0.9,1,"Davis, Albert",Black,M,23,Handgun,Yes,Yes,No,No,http://www1.cityoforlando.net/opendatadocs/saoletters/15-291159.pdf,POINT (-81.335092645 28.522822186),"LatLon(lat=28.522822186°, lon=-81.335092645°)",false,false,Gun


In [64]:
suspect_weap_cat = combine(groupby(suspects_weapon_cnt,:"Suspect's Weapon"),nrow => :count)

Row,Suspect's Weapon,count
,String31,Int64
1,Handgun,1
2,Motor Vehicle,1
3,Hands/ASP,1
4,Knife,1
5,No,1
6,"Firearm, Firearm, Unknown",1
7,"Handgun, None",1
8,Yes,1
9,Firearm,1


In [62]:
data.Match = occursin.(r"gun",data."Suspect's Weapon")
grouped = groupby(data,:Match)
grouped_cnt = combine(grouped,nrow => :Count)
grouped_cnt

Row,Match,Count
,Bool,Int64
1,false,33
2,true,45


In [40]:
pwd()

"c:\\Users\\umapa\\Documents\\git\\CatchingCrooks"

In [ ]:
#Incident Type,Number of Officers Involved 

In [63]:
Inc_type_cnt = combine(groupby(data,:"Incident Type"),nrow => :count)
sort(Inc_type_cnt,:count,rev=true)

Row,Incident Type,count
,String,Int64
1,Traffic Stop,7
2,Suspicious Person,6
3,Armed Robbery,5
4,Stolen Vehicle,4
5,Attempted Suicide,4
6,Armed Subject,4
7,Battery,3
8,Shots fired,3
9,Domestic Disturbance,2


In [ ]:
tup = (5,3,7,9,11,1)
minimum(tup)

In [ ]:
using Pkg
Pkg.add("PyCall")
Pkg.add("Conda")
Pkg.add("OpenStreetMapX")
using Conda
Conda.runconda(`install folium -c conda-forge`)

    Updating registry at `C:\Users\umapa\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\umapa\.julia\environments\v1.10\Project.toml`
  [438e738f] + PyCall v1.96.4
  No Changes to `C:\Users\umapa\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\umapa\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\umapa\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\umapa\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\umapa\.julia\environments\v1.10\Manifest.toml`
┌ Info: Running `conda install folium -c conda-forge` in root environment
└ @ Conda C:\Users\umapa\.julia\packages\Conda\zReqD\src\Conda.jl:181


Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
    current version: 24.7.1
    latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda





## Package Plan ##

  environment location: C:\Users\umapa\.julia\conda\3\x86_64

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.7.2               |     pyhd8ed1ab_0          28 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    folium-0.18.0              |     pyhd8ed1ab_0          77 KB  conda-forge
    openssl-3.4.0              |       h2466b09_0         8.1 MB  conda-forge
    xyzservices-2024.9.0       |     pyhd8ed1ab_0          46 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.4 MB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.7.2-pyhd8ed1ab_0 
  folium             conda-forge/noarch::folium-0.18.0-pyhd8ed1ab_0 
  xyzservices        conda-f

In [11]:

function convert_to_tuple(str)
    test = str[7:end]
    test = strip(test, ['(', ')'])
    test = split(test, ' ')

    return (parse(Float64, test[1]), parse(Float64, test[2]))

end

convert_to_tuple (generic function with 1 method)

In [13]:

#import/use packages and libraries here
import Pkg
Pkg.add(["OpenStreetMapX", "CSV", "DataFrames", "Geodesy", "Conda", "PyCall" ])

using Agents, OpenStreetMapX, Geodesy
using CSV, DataFrames, Conda, PyCall

Conda.runconda(`install folium -c conda-forge -y`) # Folium Map
foli = pyimport("folium")

   Resolving package versions...
   Installed CoordinateTransformations ─ v0.6.3
   Installed Geodesy ─────────────────── v1.1.0
    Updating `C:\Users\umapa\.julia\environments\v1.10\Project.toml`
  [0ef565a4] + Geodesy v1.1.0
    Updating `C:\Users\umapa\.julia\environments\v1.10\Manifest.toml`
  [150eb455] + CoordinateTransformations v0.6.3
  [0ef565a4] + Geodesy v1.1.0
Precompiling project...
  ✓ CoordinateTransformations
  ✓ Geodesy
  2 dependencies successfully precompiled in 5 seconds. 507 already precompiled.
┌ Info: Running `conda install folium -c conda-forge -y` in root environment
└ @ Conda C:\Users\umapa\.julia\packages\Conda\zReqD\src\Conda.jl:181


Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
    current version: 24.7.1
    latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda





## Package Plan ##

  environment location: C:\Users\umapa\.julia\conda\3\x86_64

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.7.2               |     pyhd8ed1ab_0          28 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    folium-0.18.0              |     pyhd8ed1ab_0          77 KB  conda-forge
    openssl-3.4.0              |       h2466b09_0         8.1 MB  conda-forge
    xyzservices-2024.9.0       |     pyhd8ed1ab_0          46 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.4 MB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.7.2-pyhd8ed1ab_0 
  folium             conda-forge/noarch::folium-0.18.0-pyhd8ed1ab_0 
  xyzservices        conda-f

PyObject <module 'folium' from 'C:\\Users\\umapa\\.julia\\conda\\3\\x86_64\\lib\\site-packages\\folium\\__init__.py'>

In [14]:
location = convert_to_tuple.(data[:, "Georeferenced Column"])

point = []

for each in location
    push!(point,LatLon(each[2],each[1]))
end

data[:, "Point"] = point

78-element Vector{Any}:
 LatLon(lat=28.559823981°, lon=-81.430522027°)
 LatLon(lat=28.426330019°, lon=-81.444407991°)
 LatLon(lat=28.497201986°, lon=-81.436934015°)
 LatLon(lat=28.574676001°, lon=-81.31615297°)
 LatLon(lat=28.552001109°, lon=-81.475974825°)
 LatLon(lat=28.552194982°, lon=-81.398586976°)
 LatLon(lat=28.596308986°, lon=-81.454123826°)
 LatLon(lat=28.540632985°, lon=-81.386192991°)
 LatLon(lat=28.522822186°, lon=-81.335092645°)
 LatLon(lat=28.497712989°, lon=-81.444618041°)
 ⋮
 LatLon(lat=28.544747996°, lon=-81.397620962°)
 LatLon(lat=28.529062019°, lon=-81.307727984°)
 LatLon(lat=28.538220076°, lon=-81.389041531°)
 LatLon(lat=28.611062074°, lon=-81.429694969°)
 LatLon(lat=28.490099922°, lon=-81.462643153°)
 LatLon(lat=28.420598013°, lon=-81.269478007°)
 LatLon(lat=28.518123007°, lon=-81.306244974°)
 LatLon(lat=28.534726006°, lon=-81.388922977°)
 LatLon(lat=28.601738987°, lon=-81.413406013°)

In [15]:
lats = [x[2] for x in location]
lons = [x[1] for x in location]
@show min_lat = minimum(lats) 
@show max_lat = maximum(lats) 
@show min_lon = minimum(lons) 
@show max_lon = maximum(lons)

map = foli.Map(
    max_bounds= true,
    location = [28.601, -81.429],
    min_lat = min_lat,
    max_lat = max_lat,
    min_lon = min_lon,
    max_lon = max_lon,
    )

    # Bounds Check
    # foli.CircleMarker([max_lat, min_lon], tooltip="Upper Left Corner").add_to(map)
    # foli.CircleMarker([min_lat, min_lon], tooltip="Lower Left Corner").add_to(map)
    # foli.CircleMarker([min_lat, max_lon], tooltip="Lower Right Corner").add_to(map)
    # foli.CircleMarker([max_lat, max_lon], tooltip="Upper Right Corner").add_to(map)

for (idx,each) in enumerate(data.Point)
    foli.Marker(
        location=[each.lat, each.lon],  # Access latitude and longitude using .lat and .lon
        tooltip="Crime Location",
        popup="Case #: $(data[!,"Case #"][idx])\n
               Inc. Type: $(data[!,"Incident Type"][idx])
              ",
        icon=foli.Icon(color="green", icon="flag")
    ).add_to(map)
end

map

     

min_lat = minimum(lats) = 28.337043705
max_lat = maximum(lats) = 28.611062074
min_lon = minimum(lons) = -81.476621992
max_lon = maximum(lons) = -81.239710013


PyObject <folium.folium.Map object at 0x000002F1882F7D60>